In [1]:
from lxml import etree
import json
import os

In [2]:
def get_initial_structure(file, lines=100):
    with open(file, 'r') as f:
        for i in range(lines):
            print(f.readline())

get_initial_structure('Posts.xml')

<?xml version="1.0" encoding="utf-8"?>

<posts>

  <row Id="4" PostTypeId="1" AcceptedAnswerId="7" CreationDate="2008-07-31T21:42:52.667" Score="794" ViewCount="70633" Body="&lt;p&gt;I want to assign the decimal variable &amp;quot;trans&amp;quot; to the double variable &amp;quot;this.Opacity&amp;quot;.&lt;/p&gt;&#xA;&lt;pre class=&quot;lang-cs prettyprint-override&quot;&gt;&lt;code&gt;decimal trans = trackBar1.Value / 5000;&#xA;this.Opacity = trans;&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&lt;p&gt;When I build the app it gives the following error:&lt;/p&gt;&#xA;&lt;blockquote&gt;&#xA;&lt;p&gt;Cannot implicitly convert type decimal to double&lt;/p&gt;&#xA;&lt;/blockquote&gt;&#xA;" OwnerUserId="8" LastEditorUserId="16124033" LastEditorDisplayName="Rich B" LastEditDate="2022-09-08T05:07:26.033" LastActivityDate="2022-09-08T05:07:26.033" Title="How to convert Decimal to Double in C#?" Tags="&lt;c#&gt;&lt;floating-point&gt;&lt;type-conversion&gt;&lt;double&gt;&lt;decimal&gt;" AnswerCount="14" Com

In [28]:
ibm_products_tag = ["java","python"]

In [29]:
from lxml import etree
import re
from tqdm import tqdm
import html
from bs4 import BeautifulSoup
from datetime import datetime
# Use a dictionary to match questions with accepted answers
accepted_answers = {}
questions_and_responses = {}

In [30]:
def process_element(elem):
    post_type = elem.get('PostTypeId')
    if post_type == '1':  # This is a question post
        tags = elem.get('Tags', '').lower()
        if 'java' in tags or 'python' in tags:
            # Only store question info and accepted answer ID
            questions_and_responses[elem.get('Id')] = {
                'question': elem.get('Title'),
                'context': elem.get('Body'),
                'accepted_answer_id': elem.get('AcceptedAnswerId'),
                'tags': tags
            }
    elif post_type == '2':  # This is a response post
        parent_id = elem.get('ParentId')
        if parent_id in questions_and_responses:
            if elem.get('Id') == questions_and_responses[parent_id]['accepted_answer_id']:
                # Store only the accepted response
                questions_and_responses[parent_id]['accepted_answer'] = elem.get('Body')

    # Clear the element to free memory
    elem.clear()

def parse_large_xml(file):
    context = etree.iterparse(file, events=('end',), tag='row')

    for event, elem in context:
        if elem.tag == 'row':
            process_element(elem)

        # Clear the element and its references to free up memory
        for ancestor in elem.xpath('ancestor-or-self::*'):
            while ancestor.getprevious() is not None:
                del ancestor.getparent()[0]

def remove_tags_except_code_and_a(body):
    soup = BeautifulSoup(body, features="html.parser")
    for tag in soup.find_all(True):
        if tag.name not in ['code', 'a']:
            tag.replace_with(tag.text)
    return str(soup)


In [31]:
parse_large_xml('Posts.xml')

In [37]:
with open('stack_overflow_python_java.json', 'w') as fp:
    json.dump(questions_and_responses, fp, indent=4)

In [3]:
def get_initial_structure(file, lines=100):
    with open(file, 'r') as f:
        for i in range(lines):
            print(f.readline())

get_initial_structure('stack_overflow_python_java.json')

{

    "123": {

        "question": "Java lib or app to convert CSV to XML file?",

        "context": "<p>Is there an existing application or library in <em>Java</em> which will allow me to convert a <code>CSV</code> data file to <code>XML</code> file?  </p>\n\n<p>The <code>XML</code> tags would be provided through possibly the first row containing column headings.</p>\n",

        "accepted_answer_id": "183",

        "tags": "<java><xml><csv><data-conversion>",

        "accepted_answer": "<p>Maybe this might help: <a href=\"http://jsefa.sourceforge.net/quick-tutorial.html\" rel=\"noreferrer\">JSefa</a></p>\n\n<p>You can read CSV file with this tool and serialize it to XML.</p>\n"

    },

    "126": {

        "question": "How would you access Object properties from within an object method?",

        "context": "<p>What is the \"purist\" or \"correct\" way to access an object's properties from within an object method that is not a getter/setter method?</p>\n\n<p>I know that from 

## Match Question and Accepted Answer

In [100]:

def process_element(elem):
    post_type = elem.get('PostTypeId')
    if post_type == '1':
        # This is a question post
        tags = elem.get('Tags', '')
        # if re.search(r'<.*\<ibm.*>', tags, re.IGNORECASE):
        accepted_answer_id = elem.get('AcceptedAnswerId')
        if accepted_answer_id is not None:
            # Add this question to our dictionary
            accepted_answers[accepted_answer_id] = {'question': elem.get('Title')}
            accepted_answers[accepted_answer_id]['tags'] = tags
    elif post_type == '2':
        # This is an answer post
        answer_id = elem.get('Id')
        if answer_id in accepted_answers:
            accepted_answers[answer_id]['answer'] = elem.get('Body')

    # Clear the element to free memory
    elem.clear()

def parse_large_xml(file):
    context = etree.iterparse(file, events=('end', ), tag='row')
    for event, elem in tqdm(context):
        process_element(elem)
        # Also eliminate now-empty references from the root node to elem
        for ancestor in elem.xpath('ancestor-or-self::*'):
            while ancestor.getprevious() is not None:
                del ancestor.getparent()[0]
    del context

def remove_tags_except_code_and_a(body):
    soup = BeautifulSoup(body, features="html.parser")
    for tag in soup.find_all(True):
        if tag.name not in ['code', 'a']:
            tag.replace_with(tag.text)
    return str(soup)

In [ ]:
accepted_answers = {}

In [101]:
parse_large_xml('Posts.xml')

58329356it [28:51, 33688.58it/s]


In [105]:
with open('stack_overflow_QApair.json', 'w') as fp:
    json.dump(accepted_answers, fp, indent=4)

In [2]:
import ijson

filename = "stack_overflow_QApair.json"
with open(filename, 'r') as f:
    count = 0
    objects = ijson.items(f, '')
    for obj in objects:
        count = len(obj)
print("Number of elements in the JSON: ", count)

Number of elements in the JSON:  11973572
